In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!pip install transformers datasets accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset, Dataset
import json
import torch

In [ ]:
with open('/content/S3_flan_old.json') as f:
    raw_data = json.load(f)

In [ ]:
!pip install transformers datasets torch

In [ ]:
import json
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq

In [ ]:
model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)

In [16]:
print(tokenizer.tokenize("{"))
print(tokenizer.tokenize("}"))

['{']
['}']


In [ ]:
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import Dataset
import json

# ✅ Load your dataset
with open("/content/drive/MyDrive/Changai/S3/Datasets/S3_flan_old.json") as f:
    raw_data = json.load(f)

INSTRUCTION = "Generate the correct Frappe query for the given question, using the provided doctype and fields."

# ✅ Prepare prompt-based data
processed_data = []
for entry in raw_data:
    try:
        input_data = entry["input"]
        if not isinstance(input_data, dict):
            continue

        prompt = (
            f"Instruction: {INSTRUCTION}\n"
            f"Doctype: {input_data['doctype']}\n"
            f"Question: {input_data['question']}\n"
            f"Fields: {input_data['fields']}"
        )

        processed_data.append({
            "input": prompt,
            "output": entry["output"]
        })
    except Exception as e:
        print("Skipping due to error:", e)
        continue

# ✅ Convert to Hugging Face Dataset
dataset = Dataset.from_list(processed_data)
dataset = dataset.train_test_split(test_size=0.1)


model_name = "google/flan-t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)

# ✅ Add curly braces as special tokens (ONLY if they map to <unk>)
special_tokens_dict = {'additional_special_tokens': ['{', '}']}
num_added = tokenizer.add_special_tokens(special_tokens_dict)

# ✅ Load and resize model to accommodate new tokens
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

# ✅ Tokenize with `{}` token loss reinforcement
def tokenize_and_enhance_loss(example):
    model_inputs = tokenizer(example["input"], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["output"], max_length=128, truncation=True, padding="max_length")

    # Reinforce loss on `{` and `}`
    target_ids = labels["input_ids"]
    for i, token_id in enumerate(target_ids):
        if token_id == tokenizer.pad_token_id:
            target_ids[i] = -100  # Mask pad tokens
        if token_id in [tokenizer.convert_tokens_to_ids('{'), tokenizer.convert_tokens_to_ids('}')]:
            continue  # Keep loss for curly braces

    model_inputs["labels"] = target_ids
    return model_inputs

# ✅ Tokenize dataset
tokenized_datasets = dataset.map(tokenize_and_enhance_loss, batched=False)

# ✅ Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Changai/S3_test",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=10,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100
)

# ✅ Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ✅ Trainer setup
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# ✅ Train and Save
trainer.train()
trainer.evaluate()
model.save_pretrained("/content/drive/MyDrive/Changai/S3_test")
tokenizer.save_pretrained("/content/drive/MyDrive/Changai/S3_test")

In [17]:
tokenizer.add_special_tokens({'additional_special_tokens': ['{', '}']})
model.resize_token_embeddings(len(tokenizer))

Embedding(32102, 768)

In [6]:
from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from datasets import Dataset
import json

# ✅ Load your dataset
with open("/content/all_data_s3.json") as f:
    raw_data = json.load(f)

INSTRUCTION = "Generate the correct Frappe query for the given question, using the provided doctype and fields."

# ✅ Prepare prompt-based data
processed_data = []
for entry in raw_data:
    try:
        input_data = entry["input"]
        if not isinstance(input_data, dict):
            continue

        prompt = (
            f"Instruction: {INSTRUCTION}\n"
            f"Doctype: {input_data['doctype']}\n"
            f"Question: {input_data['question']}\n"
            f"Fields: {input_data['fields']}"
        )

        processed_data.append({
            "input": prompt,
            "output": entry["output"]
        })
    except Exception as e:
        print("Skipping due to error:", e)
        continue

# ✅ Convert to Hugging Face Dataset
dataset = Dataset.from_list(processed_data)
dataset = dataset.train_test_split(test_size=0.1)



model_path = "hyrinmansoor/text2frappe-s3-flan-query"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# ✅ Add curly braces as special tokens (ONLY if they map to <unk>)
# special_tokens_dict = {'additional_special_tokens': ['{', '}']}
# num_added = tokenizer.add_special_tokens(special_tokens_dict)

# ✅ Load and resize model to accommodate new tokens
# model = T5ForConditionalGeneration.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

# ✅ Tokenize with `{}` token loss reinforcement
def tokenize_and_enhance_loss(example):
    model_inputs = tokenizer(example["input"], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["output"], max_length=128, truncation=True, padding="max_length")


    # Reinforce loss on `{` and `}`
    target_ids = labels["input_ids"]
    for i, token_id in enumerate(target_ids):
        if token_id == tokenizer.pad_token_id:
            target_ids[i] = -100  # Mask pad tokens
        if token_id in [tokenizer.convert_tokens_to_ids('{'), tokenizer.convert_tokens_to_ids('}')]:
            continue  # Keep loss for curly braces

    model_inputs["labels"] = target_ids
    return model_inputs

# ✅ Tokenize dataset
tokenized_datasets = dataset.map(tokenize_and_enhance_loss, batched=False)

# ✅ Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Changai/S3/Model/S3_flan_test_all_data",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=10,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=100
)

# ✅ Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ✅ Trainer setup
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# ✅ Train and Save
trainer.train()
trainer.evaluate()
model.save_pretrained("/content/drive/MyDrive/Changai/S3/Model/S3_flan_test_all_data")
tokenizer.save_pretrained("/content/drive/MyDrive/Changai/S3/Model/S3_flan_test_all_data")

Skipping due to error: 'fields'
Skipping due to error: 'fields'
Skipping due to error: 'fields'


Map:   0%|          | 0/4788 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/533 [00:00<?, ? examples/s]

/tmp/ipython-input-6-2497874914.py:97: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hyrin (hyrin-erpgulf) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.106600,0.073154
2,0.092600,0.072000
3,0.077700,0.073799
4,0.088800,0.070477
5,0.052800,0.068671
6,0.063000,0.071015
7,0.058000,0.070256
8,0.042300,0.073597
9,0.055400,0.071515
10,0.050400,0.071294


('/content/drive/MyDrive/Changai/S3/Model/S3_flan_test_all_data/tokenizer_config.json',
 '/content/drive/MyDrive/Changai/S3/Model/S3_flan_test_all_data/special_tokens_map.json',
 '/content/drive/MyDrive/Changai/S3/Model/S3_flan_test_all_data/spiece.model',
 '/content/drive/MyDrive/Changai/S3/Model/S3_flan_test_all_data/added_tokens.json')

In [22]:
print(tokenizer.convert_tokens_to_ids(['<', '>','!=','<=','>=','=','!','{', '}']))

[2, 3155, 2, 2, 2, 2423, 55, 32100, 32101]


In [36]:
tokenizer.add_special_tokens({
    'additional_special_tokens': ['<', '>', '!=', '<=', '>=', '=', '!', '{', '}']
})
model.resize_token_embeddings(len(tokenizer))


Embedding(32108, 768)

In [44]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load tokenizer and model
model_path = "/content/drive/MyDrive/Changai/S3/Model/S3_flan_test_all_data"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)
prompts=[
  {
    "doctype": "Contact",
    "question": "List all contacts with the first name 'Fatima'.",
    "fields": ["first_name"]
  },
  {
    "doctype": "Contact",
    "question": "Find contacts with email ID 'ali@example.com'.",
    "fields": ["email_id"]
  },
  {
    "doctype": "Contact",
    "question": "Show contacts whose phone number starts with '+974'.",
    "fields": ["phone"]
  },
  {
    "doctype": "Contact",
    "question": "Get contacts with the status 'Lead'.",
    "fields": ["status"]
  },
  {
    "doctype": "Contact",
    "question": "Find female contacts who are primary contacts.",
    "fields": ["gender", "is_primary_contact"]
  },
  {
    "doctype": "Contact",
    "question": "List all contacts marked as billing contacts.",
    "fields": ["is_billing_contact"]
  },
  {
    "doctype": "Contact",
    "question": "Find all contacts with both phone and mobile numbers.",
    "fields": ["phone", "mobile_no"]
  },
  {
    "doctype": "Contact",
    "question": "Show contacts who belong to the 'Marketing' department.",
    "fields": ["department"]
  },
  {
    "doctype": "Contact",
    "question": "Get contacts from company name containing 'Global Solutions'.",
    "fields": ["company_name"]
  },
  {
    "doctype": "Contact",
    "question": "Which contacts are linked to ERP user 'sara@acme.com'?",
    "fields": ["user"]
  },
  {
    "doctype": "Contact",
    "question": "Find contacts that are subscribed and synced with Google.",
    "fields": ["unsubscribed", "sync_with_google_contacts"]
  },
  {
    "doctype": "Contact",
    "question": "List contacts having a designation of 'Sales Executive' and gender 'Male'.",
    "fields": ["designation", "gender"]
  },
  {
    "doctype": "Contact",
    "question": "Get contacts that have uploaded a profile image.",
    "fields": ["image"]
  },
  {
    "doctype": "Contact",
    "question": "Find all contacts with middle name 'Hassan' and last name 'Ali'.",
    "fields": ["middle_name", "last_name"]
  },
  {
    "doctype": "Contact",
    "question": "Which contacts have full name 'Ahmed bin Saeed' and are not unsubscribed?",
    "fields": ["full_name", "unsubscribed"]
  }
]


for idx, item in enumerate(prompts, 1):
    test_prompt = f"""
Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: {item['doctype']}
Question: {item['question']}
Fields: {item['fields']}
"""
    inputs = tokenizer(test_prompt.strip(), return_tensors="pt")
    output_ids = model.generate(
        input_ids=inputs["input_ids"],
        max_length=128,
        do_sample=False,
        num_beams=4,
        early_stopping=True
    )
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)

    output_text = output_text.replace("<pad>", "").replace("</s>", "").strip()

    print(f"\n🔢 Example {idx}")
    print("🗒️ Prompt:\n", test_prompt.strip())
    print("📄 Generated Query:\n", output_text)



🔢 Example 1
🗒️ Prompt:
 Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Contact
Question: List all contacts with the first name 'Fatima'.
Fields: ['first_name']
📄 Generated Query:
 frappe.get_list('Contact', filters= { 'first_name': 'Fatima' } , fields=['name', 'first_name'])

🔢 Example 2
🗒️ Prompt:
 Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Contact
Question: Find contacts with email ID 'ali@example.com'.
Fields: ['email_id']
📄 Generated Query:
 frappe.get_all('Contact', filters= { 'email_id': 'ali@example.com' } , fields=['name', 'email_id'])

🔢 Example 3
🗒️ Prompt:
 Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Contact
Question: Show contacts whose phone number starts with '+974'.
Fields: ['phone']
📄 Generated Query:
 frappe.get_all('Contact', filters= { 'phone': ['

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_path = "/content/drive/MyDrive/Changai/S3_test"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# 🔍 Sample test input
test_prompt = """Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Company
Question: Show me company details where the name is 'France'.
Fields: name, country"""

inputs = tokenizer(test_prompt, return_tensors="pt")

output_ids = model.generate(
    input_ids=inputs["input_ids"],
    max_length=128,
    do_sample=False,
    num_beams=4,
    early_stopping=True
)

# ✅ Decode and print output
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=False)
output_text=output_text.replace("<pad>","")
output_text=output_text.replace("</s>","")
print("📄 Generated Query:\n", output_text)


In [ ]:
model_path="/content/drive/MyDrive/Changai/S3_test"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

In [4]:
from huggingface_hub import HfApi, HfFolder, notebook_login
notebook_login()

In [ ]:
from huggingface_hub import create_repo

create_repo("text2frappe-s3-flan-query", private=True)

In [ ]:
from huggingface_hub import upload_folder

upload_folder(
    repo_id="hyrinmansoor/text2frappe-s3-flan-query",
    folder_path=model_path,
    path_in_repo=".",
    repo_type="model"
)


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained('hyrinmansoor/text2frappe-s3-flan-query')
tokenizer = T5Tokenizer.from_pretrained('hyrinmansoor/text2frappe-s3-flan-query')
test_cases = [
    """Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Company
Fields: company_name
Question: What's the full name of our primary registered company named 'HTS Pvt Ltd'?""",

    """Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Company
Fields: parent_company
Question: Who is listed as the parent company of 'FusionCorp International'?""",

    """Instruction: Generate the correct Frappe query for the given question, using the provided doctype and fields.
Doctype: Finance Book
Fields: finance_book_name
Question: Can I rename or update the name of an existing finance book?"""
]

# Define generation function
def generate_query(test_input):
    inputs = tokenizer(test_input, return_tensors="pt", max_length=512, truncation=True, padding=True)
    outputs = model.generate(inputs['input_ids'], max_length=512, num_beams=5, early_stopping=True)
    decoded= tokenizer.decode(outputs[0], skip_special_tokens=False)
    decoded=output_text.replace("<pad>","")
    decoded=output_text.replace("</s>","")
    return decoded


# Run test cases
for i, test_input in enumerate(test_cases, start=1):
    print(f"\n--- Test Case {i} ---")
    print(f"Input:\n{test_input}")
    print("\nGenerated Query:")
    print(generate_query(test_input))
    print("\n")
